Check that the 3 Grover-compatible retractions in the paper meet the definition.

In [31]:
import numpy as np

# ----------------- basic linear-algebra helpers ----------------- #

def random_unitary(n: int) -> np.ndarray:
    """Generate a random Haar-like unitary via QR."""
    A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, R = np.linalg.qr(A)
    d = np.diag(R)
    ph = d / np.abs(d)
    Q = Q * ph
    return Q

def projector_from_vector(v: np.ndarray) -> np.ndarray:
    """Rank-1 projector |v><v| with ||v||=1."""
    v = v / np.linalg.norm(v)
    return np.outer(v, np.conjugate(v))

def exp_i_projector(theta: float, P: np.ndarray) -> np.ndarray:
    """e^{i theta P} for a projector P (P^2=P)."""
    n = P.shape[0]
    return np.eye(n, dtype=complex) + (np.exp(1j * theta) - 1.0) * P

def fro_norm(A: np.ndarray) -> float:
    return np.linalg.norm(A, "fro") # type: ignore

def comm(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    return A @ B - B @ A

def generate_H_psi0(N: int, tol: float = 1e-3, max_tries: int = 1000):
    """
    Generate a random projector H and a pure state psi0_vec
    such that q0 = <psi0|H|psi0> is strictly between tol and 1 - tol.
    """
    for _ in range(max_tries):
        h = np.random.randn(N) + 1j * np.random.randn(N)
        h /= np.linalg.norm(h)
        H = projector_from_vector(h)

        psi0_vec = np.random.randn(N) + 1j * np.random.randn(N)
        psi0_vec /= np.linalg.norm(psi0_vec)

        q0 = np.vdot(psi0_vec, H @ psi0_vec).real
        if tol < q0 < 1 - tol:
            P0 = projector_from_vector(psi0_vec)
            return H, psi0_vec, P0, q0
    raise RuntimeError("Failed to find suitable psi0 within max_tries.")


# ----------------- Grover data: X0, Y0, W ----------------- #

def build_grover_data(N: int):
    """
    Construct H, psi0, P0, q0, and the basis X0, Y0 for W = span_R{X0, Y0}.
    """
    H, psi0_vec, P0, q0 = generate_H_psi0(N)
    X0 = comm(H, P0)
    Y0 = 1j * comm(H, X0)

    print("q0 =", q0)
    print("Lemma check: ||X0||_F, ||Y0||_F =", fro_norm(X0), fro_norm(Y0))
    print("Lemma check: <X0,Y0> =", np.trace(X0.conj().T @ Y0))
    return H, psi0_vec, P0, q0, X0, Y0



# 8 factor retraction

Define
\begin{equation*}
    \mathrm{R}^{(8)}_U (\eta):=e^{i \frac{y}{2}  \psi_0} e^{i \frac{\pi}{2} H} e^{-i \frac{x}{2}  \psi_0} e^{-i \pi H} e^{i \frac{x}{2}  \psi_0} e^{-i \frac{\pi}{2} H} e^{-i \frac{y}{2}  \psi_0} e^{i \pi H} \, U.
\end{equation*}
We can show that $\gamma (t)=\mathrm{R}^{(8)}_U (t\eta)$ satisfies $\gamma (0)=U$ and $\dot{\gamma} (0)=\eta \in \mathcal{W} U$.

In [32]:

def R8_path(U, H, P0, x, y, t):
    """
    8-factor retraction path gamma(t) = R_U^{(8)}(t eta).
    eta tilde = x X0 + y Y0; here x,y are fixed and we scale them by t.
    """
    xt = t * x
    yt = t * y

    E_H_pihalf     = exp_i_projector(np.pi/2, H)
    E_H_minuspi    = exp_i_projector(-np.pi,  H)
    E_H_minus_half = exp_i_projector(-np.pi/2, H)
    E_H_pi         = exp_i_projector(np.pi,   H)

    E_P_y2         = exp_i_projector( yt/2.0,  P0)
    E_P_minus_y2   = exp_i_projector(-yt/2.0,  P0)
    E_P_x2         = exp_i_projector( xt/2.0,  P0)
    E_P_minus_x2   = exp_i_projector(-xt/2.0,  P0)

    V = (
        E_P_y2
        @ E_H_pihalf
        @ E_P_minus_x2
        @ E_H_minuspi
        @ E_P_x2
        @ E_H_minus_half
        @ E_P_minus_y2
        @ E_H_pi
    )
    return V @ U


# 6 factor retraction

Let
\begin{equation*}
    c_1: =-\frac{x+y}{2}, \quad c_2: =\frac{x-y}{2},
\end{equation*}
and define
\begin{equation*}
    \mathrm{R}^{(6)}_U (\eta): =e^{i \frac{\pi}{2} H} e^{i c_1 \psi_0} e^{-i \pi H} e^{i c_2 \psi_0} e^{i \frac{\pi}{2} H} e^{i y t \psi_0} \, U.
\end{equation*}
We can show that $\gamma (t)=\mathrm{R}^{(6)}_U (t\eta)$ satisfies $\gamma (0)=U$ and $\dot{\gamma} (0)=\eta \in \mathcal{W} U$.

In [33]:


def R6_path(U, H, P0, x, y, t):
    """
    6-factor retraction path gamma(t) = R_U^{(6)}(t eta).
    Here we use c1 = -(x+y)/2, c2 = (x-y)/2 and scale by t.
    The last factor uses angle y_t = t*y; the extra 't' in the paper
    is treated as a typo.
    """
    c1 = -(x + y) / 2.0
    c2 = (x - y) / 2.0
    c1_t = t * c1
    c2_t = t * c2
    y_t  = t * y

    V = (
        exp_i_projector(np.pi/2, H)
        @ exp_i_projector(c1_t, P0)
        @ exp_i_projector(-np.pi, H)
        @ exp_i_projector(c2_t, P0)
        @ exp_i_projector(np.pi/2, H)
        @ exp_i_projector(y_t, P0)
    )
    return V @ U


# 5 factor retraction
Let
\begin{equation}
    A: =\arctan 2 (y, x), \quad R: =\sqrt{x^2+y^2},
\end{equation}
(the argument and modulus of the complex number $x+i y$) and set
\begin{equation} 
    a_1=A+\frac{\pi}{2},  \quad a_2=A-\frac{\pi}{2},  \quad b_1=-\frac{R}{2},   \quad b_2= \frac{R}{2}.
\end{equation}
Define
\begin{equation} 
    \mathrm{R}^{(5)}_U (\eta): =e^{i a_1 H} e^{i b_1 \psi_0} e^{i\left (a_2-a_1\right) H} e^{i b_2 \psi_0} e^{-i a_2 H} \, U.
\end{equation}
Since $a_2-a_1=-\pi$, the middle $H$-factor simplifies to $e^{-i \pi H}$. Then
\begin{equation} 
\gamma (t)=\mathrm{R}^{(5)}_U (t\eta)=\underbrace{e^{i a_1 H} e^{i t b_1 \psi_0} e^{i\left(a_2-a_1\right) H} e^{i t b_2 \psi_0} e^{-i a_2 H}}_{\text {newly added gates } V(t ):=} \, U, \quad t \geq 0.
\end{equation}
We can show that $\gamma (t)$ satisfies $\gamma (0)=U$ and $\dot{\gamma} (0)=\eta \in \mathcal{W} U$.

In [34]:


def R5_path(U, H, P0, x, y, t):
    """
    5-factor retraction path gamma(t) = R_U^{(5)}(t eta).
    A = atan2(y,x), R = sqrt(x^2+y^2) (polar coordinates of x+iy).
    """
    A = np.arctan2(y, x)
    R = np.hypot(x, y)

    a1 = A + np.pi/2
    a2 = A - np.pi/2
    b1 = -R / 2.0
    b2 =  R / 2.0

    V = (
        exp_i_projector(a1, H)
        @ exp_i_projector(t * b1, P0)
        @ exp_i_projector(a2 - a1, H)  # equals e^{-i pi H}
        @ exp_i_projector(t * b2, P0)
        @ exp_i_projector(-a2, H)
    )
    return V @ U



In [35]:

# ----------------- numerical verification of Definition 3.4 ----------------- #

def check_retraction(R_path, name, H, P0, X0, Y0, N, trials=20, dt=1e-6):
    """
    Numerically verify that for eta in W U the map R_path satisfies:
        (i)  R_U(0 * eta) = U
        (ii) d/dt|_{t=0} R_U(t eta) = eta          (finite difference)
        (iii) R_U(t eta) is unitary for t=1
    """
    print(f"\n=== Testing {name} retraction ===")
    max_base_err = 0.0
    max_deriv_err = 0.0

    for _ in range(trials):
        U = random_unitary(N)
        # choose real coefficients (x,y) so that eta tilde lies in W
        x, y = np.random.randn(), np.random.randn()
        tilde_eta = x * X0 + y * Y0    # skew-Hermitian in W
        eta = tilde_eta @ U            # tangent vector at U

        # (i) R_U(0 * eta) = U
        U0 = R_path(U, H, P0, x, y, 0.0)
        base_err = fro_norm(U0 - U)
        max_base_err = max(max_base_err, base_err)

        # (ii) derivative at 0 equals eta
        U_plus = R_path(U, H, P0, x, y, dt)
        deriv = (U_plus - U0) / dt
        deriv_err = fro_norm(deriv - eta)
        max_deriv_err = max(max_deriv_err, deriv_err)

    print(f"max ||R_U(0)-U||_F       = {max_base_err:.3e}")
    print(f"max derivative error     = {max_deriv_err:.3e}  (dt = {dt})")
    print("Suggested tolerances: base, unitarity ~ 1e-10; derivative ~ 1e-4.")


# ----------------- main: run the checks ----------------- #

if __name__ == "__main__":
    # np.random.seed(0)

    # dimension of Hilbert space
    N = 4

    # build H, psi0, X0, Y0
    H, psi0_vec, P0, q0, X0, Y0 = build_grover_data(N)

    # check the three Grover-compatible retractions
    check_retraction(R8_path, "8-factor", H, P0, X0, Y0, N)
    check_retraction(R6_path, "6-factor", H, P0, X0, Y0, N)
    check_retraction(R5_path, "5-factor", H, P0, X0, Y0, N)


q0 = 0.1724575186853366
Lemma check: ||X0||_F, ||Y0||_F = 0.5342582202160926 0.5342582202160925
Lemma check: <X0,Y0> = (-5.833112032700608e-18+6.938893903907228e-18j)

=== Testing 8-factor retraction ===
max ||R_U(0)-U||_F       = 4.212e-16
max derivative error     = 1.290e-06  (dt = 1e-06)
Suggested tolerances: base, unitarity ~ 1e-10; derivative ~ 1e-4.

=== Testing 6-factor retraction ===
max ||R_U(0)-U||_F       = 4.314e-16
max derivative error     = 1.442e-06  (dt = 1e-06)
Suggested tolerances: base, unitarity ~ 1e-10; derivative ~ 1e-4.

=== Testing 5-factor retraction ===
max ||R_U(0)-U||_F       = 4.458e-16
max derivative error     = 8.431e-07  (dt = 1e-06)
Suggested tolerances: base, unitarity ~ 1e-10; derivative ~ 1e-4.
